# Multitask learning notebook

In this notebook we'll do some toy experiments with a multitask learning model.

In [1]:
import csv
import torch.optim as opt
from torch.nn import CrossEntropyLoss
from mlearn.utils.pipeline import process_and_batch
from mlearn.utils.metrics import Metrics
from mlearn.utils.train import train_mtl_model, run_mtl_model
from mlearn.data.dataset import GeneralDataset
from mlearn.data.clean import Preprocessors, Cleaner
from mlearn.data.loaders import hoover, davidson, garcia, semeval_sentiment
from mlearn.modeling.multitask import EmbeddingLSTMClassifier, OnehotLSTMClassifier, OnehotMLPClassifier

## Load the datasets to be used

In [2]:
cleaner = Cleaner(['lower', 'hashtag', 'url', 'user'])
data_path = '../tests/data/'
preprocessr = Preprocessors()

# Hoover dataset
mftc = hoover(cleaners = cleaner, data_path = data_path)
mftc.build_token_vocab(mftc.data)
mftc.build_label_vocab(mftc.data)

# Davidson dataset
off = davidson(cleaners = cleaner, data_path = data_path)
off.build_token_vocab(off.data)
off.build_label_vocab(off.data)

# Sentiment analysis dataset
sent = semeval_sentiment(cleaners = cleaner, data_path = data_path)
sent.build_token_vocab(sent.data)
sent.build_label_vocab(sent.data)

Loading Hoover et al. (train): 999it [00:04, 220.93it/s]
Encode label vocab: 100%|██████████| 11/11 [00:00<00:00, 990.11it/s]
Loading Davidson et al. (train): 887it [00:04, 212.44it/s]
Encode label vocab: 100%|██████████| 3/3 [00:00<00:00, 454.96it/s]
Loading Semeval (2016) (train): 999it [00:04, 204.59it/s]
Encode label vocab: 100%|██████████| 3/3 [00:00<00:00, 3405.39it/s]


## Create Batches of train and dev

## Initialize models

In [3]:
input_dims = [sent.vocab_size(), off.vocab_size(), mftc.vocab_size()]
output_dims = [sent.label_count(), off.label_count(), mftc.label_count()]
shared_dim, hidden_dims, embedding_dim = 256, [64, 128, 300], 128
dropout = 0.4
print(input_dims)

[3476, 2915, 2823]


In [4]:
ohmlp = OnehotMLPClassifier(input_dims = input_dims, shared_dim = shared_dim, hidden_dims = hidden_dims,
                            output_dims = output_dims, dropout = dropout)
optim = opt.SGD(ohmlp.parameters(), lr = 0.01)
loss = CrossEntropyLoss()
m = Metrics(['f1-score'], display_metric = 'f1-score', early_stop = 'loss')
dev_m = Metrics(['f1-score', 'accuracy'], display_metric = 'f1-score', early_stop = 'loss')
batchers = [process_and_batch(dataset, dataset.data, 32) for dataset in [sent, off, mftc]]

OnehotMLPClassifier(
  (all_parameters): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 64x3476]
      (1): Parameter containing: [torch.FloatTensor of size 64]
      (2): Parameter containing: [torch.FloatTensor of size 64x2915]
      (3): Parameter containing: [torch.FloatTensor of size 64]
      (4): Parameter containing: [torch.FloatTensor of size 64x2823]
      (5): Parameter containing: [torch.FloatTensor of size 64]
      (6): Parameter containing: [torch.FloatTensor of size 128x64]
      (7): Parameter containing: [torch.FloatTensor of size 128]
      (8): Parameter containing: [torch.FloatTensor of size 300x128]
      (9): Parameter containing: [torch.FloatTensor of size 300]
      (10): Parameter containing: [torch.FloatTensor of size 3x300]
      (11): Parameter containing: [torch.FloatTensor of size 3]
      (12): Parameter containing: [torch.FloatTensor of size 3x300]
      (13): Parameter containing: [torch.FloatTensor of size 3]
      (14): Pa

In [5]:
writer = csv.writer(open('testing', 'w', encoding = 'utf-8'), delimiter = '\t')
base = ['timestamp', 'Trained On', 'Predicted On']
model_hdr = ['Epoch', 'Model', 'Input dim', 'Hidden dim', 'Embedding dim', 'Dropout', 'Learning rate', 
             'Window sizes', 'Num filters', 'Max feats', 'Output dim']
hyper_hdr = ['# Epochs', 'Learning rate', 'Batch size']
metric_hdr = [metric for metric in list(m.scores.keys()) + [f'Dev {m}' for m in dev_m.scores]]
writer.writerow(base + model_hdr + hyper_hdr + metric_hdr)

setattr(m, 'tasks', [])
hyper_info = [20, 0.01, 32]

In [6]:
run_mtl_model(train = True, writer = writer, epochs = 20, model = ohmlp, loss_f = loss, metrics = m, 
              batchers = batchers, opt = optim, save_path = 'testings', dev = process_and_batch(sent, sent.dev, len(sent.dev)),
              dev_metrics = dev_m, model_hdr = model_hdr, hyper_info = hyper_info, main_name = sent.name, 
              earlystop = 2, batches_per_epoch = 60,
              data_name = 'test', metric_hdr = metric_hdr, gpu = False, store = False)

Batch:   0%|          | 0/60 [00:00<?, ?it/s]

60


/Users/zeerakw/.virtualenvs/mlearn/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)

Batch:  50%|█████     | 30/60 [00:07<00:06,  4.90it/s, batch_loss=0.0321, diff=0.408, epoch_loss=0.0431, f1-score=0.259, task=1]


Batch: 100%|██████████| 60/60 [00:11<00:00,  7.70it/s, batch_loss=0.0324, diff=0.364, epoch_loss=0.0461, f1-score=0.272, task=0]
                                                                                                                                
Evaluating model: 100%|██████████| 1/1 [00:00<00:00,  3.73it/s]


                                                               /Users/zeerakw/.virtualenvs/mlearn/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
Batch:   2%|▏         | 1/60 [00:00<00:08,  7.36it/s, batch_loss=0.0299, diff=0.274, epoch_loss=0.0299, f1-score=0.28, task=1]

...

Batch: 100%|██████████| 60/60 [00:08<00:00,  6.36it/s, batch_loss=0.0252, diff=0.465, epoch_loss=0.0403, f1-score=0.341, task=1]
                                                                                                                                
Batch:   2%|▏         | 1/60 [00:00<00:09,  5.94it/s, batch_loss=0.0723, diff=-.544, epoch_loss=0.0723, f1-score=0.34, task=2]

60



Batch:  53%|█████▎    | 32/60 [00:04<00:03,  7.19it/s, batch_loss=0.0720, diff=-.554, epoch_loss=0.0453, f1-score=0.338, task=2]


Batch: 100%|██████████| 60/60 [00:09<00:00,  6.19it/s, batch_loss=0.0232, diff=0.542, epoch_loss=0.0429, f1-score=0.337, task=1]
                                                                                                                                
Training model:  35%|███▌      | 7/20 [01:30<02:47, 12.89s/it, dev_loss=0.0105, dev_score=0.6800, loss=0.0232]

Early stopping: Terminate
Loading weights from epoch 5


In [7]:
run_mtl_model(train = False, writer = writer, pred_writer = writer, model = ohmlp, loss_f = loss, metrics = m, gpu = False,
              batchers = process_and_batch(sent, sent.test, 100), mtl = 0, store = True, test = sent.test, data = sent.test,
              model_hdr = model_hdr, hyper_info = hyper_info, main_name = sent.name, data_name = 'test',
              metric_hdr = metric_hdr, dataset = sent, train_field = 'text', label_field = 'label')